In [1]:
import os
import sys
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
import torchaudio
import soundfile
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment

module_path = os.path.abspath(os.path.join('./source/'))
if module_path not in sys.path:
    sys.path.append(module_path)

from model import DiffWave
from dataset import ChunkedMusDBHQ
from train import train

c:\Users\hause\miniconda3\envs\practical_work\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
chunked_data = ChunkedMusDBHQ(audio_dir='data/chunked_audio/')

trainloader = torch.utils.data.DataLoader(
    chunked_data,
    batch_size=2,
    shuffle=True,
    )

In [3]:
variance_schedule = torch.tensor([0.0001,0.001,0.01,0.05,0.2,0.5]) #torch.linspace(0.1, 0.7, 10)
model = train(64, trainloader, 1, len(variance_schedule), variance_schedule)

epoch: 0 | step: 0 | loss: 0.6726873517036438
epoch: 0 | step: 5 | loss: 0.6573325395584106
epoch: 0 | step: 10 | loss: 0.6505427360534668
epoch: 0 | step: 15 | loss: 0.6409654021263123
Epoch DONE! epoch: 0 | loss: 0.6655532121658325


In [4]:
x_t = torch.randn(1, 1, 220500)
sample = model.sample(x_t)

### trying out stuff!

In [ ]:
#test forward pass with one sample
sample = next(iter(trainloader)) #waveform, sample_rate
waveform = sample[0]
print(waveform.shape) #batch size, num channels, sample length

#only take first channel of sample; indexing is done with 0:1 instead of [0], otherwise shape is not kept the same
waveform = waveform[:,0:1,:]

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DiffWave(32)
model = model.to(device)

In [ ]:
y = model.forward(waveform)

In [ ]:
output = y[0].detach()
output.shape

In [ ]:
path = "test.wav"
torchaudio.save(path, output, 44100)